# Introdução à Linguística Computacional

## Análise de textos jornalísticos e artigos científicos sobre enchentes na cidade de São Paulo

Por:     | 
-------- | -----
André Bernardino Cardim| 11000515
Bianca Miyata Ranieri    | 11042615
Henrique Queiroz Reuter     | 11201812261

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import requests
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from nltk.tokenize import sent_tokenize
import matplotlib
import matplotlib.pyplot as plt
nltk.download('punkt')

In [ ]:
import pdfminer
from pdfminer.high_level import extract_text

print(pdfminer.__version__)  

In [ ]:
!pip3 install newspaper3k
!pip3 install pdfminer.six

## Preparação dos arquivos .txt para jornalisticos e científicos

### Textos jornalísticos
Utilizando a biblioteca newspaper, acessar cada URL presente no arquivo 'urls_j.txt' e salvar o conteúdo do artigo jornalístico em um arquivo (número)j.txt na pasta 'textos-jornalisticos'

In [ ]:
# Importar a ferramenta
from newspaper import Article
 
#Ler cada linha do arquivo txt e colocar na lista 'urls'
with open("urls_j.txt") as file:
    urls = file.read().splitlines()

#Para cada url pegar o titulo+texto e salvar em um arquivo .txt
i=0
while(i<26):
     download and parse article
    article = Article(urls[i])
    article.download()
    article.parse()

    file = open('textos-jornalisticos/{}j.txt'.format(i+1), 'w')
    file.write(article.title+"\n")
    file.write(article.text)
    file.close()

    i = i + 1

### Textos científicos
Utilizando o pdfminer, extrair o texto de cada artigo científico no formato .pdf da pasta 'pdfs-nao-convertidos' e salvar em um arquivo (número)c.txt na pasta 'pdfs-convertidos'

In [ ]:
i = 1
while(i<=9):
    text = extract_text('pdfs-nao-convertidos/{}c.pdf'.format(i))
    file = open('pdfs-convertidos/{}c.txt'.format(i), 'w')
    file.write(text)
    file.close()
    i = i + 1

## Criação dos corpus jornalístico e científico
Utilizando o PlaintextCorpusReader, criar o corpus jornalístico e científico e mostrar 3 das principais informações deles:

In [ ]:
import os
import nltk
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords

#Corpus jornalistico

corpus_j_path= 'textos-jornalisticos/'

corpus_j = PlaintextCorpusReader(corpus_j_path, '.*')

print('O corpus jornalístico tem', len(corpus_j.fileids()), 'textos,', 
len(corpus_j.words()), 'palavras e', len(corpus_j.sents()), 'sentenças')

#Corpus científico
 
corpus_c_path= 'pdfs-convertidos/'

corpus_c = PlaintextCorpusReader(corpus_c_path, '.*')

print('O corpus científico tem', len(corpus_c.fileids()), 'textos,', 
len(corpus_c.words()), 'palavras e', len(corpus_c.sents()), 'sentenças')

Calcular a diversidade lexical de ambos os corpus:

In [ ]:
#Cálculo da diversidade lexical
def lexical_diversity(texto):
    return len(set(texto)) / len(texto)

print("Diversidade do corpus jornalístico:", 
    100*lexical_diversity(corpus_j.words()), '%')

print("Diversidade do corpus científico:",
     100*lexical_diversity(corpus_c.words()), '%')

## Análise dos corpus pelo analisador morfológico CoGrOO
Importação do biblioteca para utilização (disponibilizada pela interface CoGrOO4Py), e teste com uma frase simples:

In [ ]:
from cogroo_interface import Cogroo
cogroo = Cogroo.Instance()

Definição da lista de classficiações morfológicas válidas:

In [ ]:
classes_cogroo=['n', 'prop', 'art', 'pron', 'pron-pers', 'pron-det',
                'pron-indp', 'adj', 'n-adj', 'v', 'v-fin', 'v-inf',
                'v-pcp', 'v-ger', 'num', 'prp', 'adj', 'conj',
                'conj-s','conj-c', 'intj', 'adv', 'xxx']

Criação do dicionário pos_tags da classe Cogroo, para traduzir as classificações geradas pelo CoGrOO para um formato legível em português:

In [ ]:
def _pos_tags(self):
    pos = {}
    pos.update({"n": "substantivo"})
    pos.update({"prop": "nome próprio"})
    pos.update({"art": "artigo"})
    pos.update({"pron": "pronome"})
    pos.update({"pron-pers": "pronome pessoal"})
    pos.update({"pron-det": "pronome determinativo"})
    pos.update({"pron-indp": "substantivo/pron-indp"})
    pos.update({"adj": "adjetivo"})
    pos.update({"n-adj": "substantivo/adjetivo"})
    pos.update({"v": "verbo"})
    pos.update({"v-fin": "verbo finitivo"})
    pos.update({"v-inf": "verbo infinitivo"})
    pos.update({"v-pcp": "verbo particípio"})
    pos.update({"v-ger": "verbo gerúndio"})
    pos.update({"num": "numeral"})
    pos.update({"prp": "preposição"})
    pos.update({"adj": "adjetivo"})
    pos.update({"conj": "conjunção"})
    pos.update({"conj-s": "conjunção subordinativa"})
    pos.update({"conj-c": "conjunção coordenativa"})
    pos.update({"intj": "interjeição"})
    pos.update({"adv": "advérbio"})
    pos.update({"xxx": "outro"})
    return pos

# pos: "part of speech"
pos = cogroo.pos_tags

### Análise morfológica do corpus jornalístico
Análise do cogroo do corpus jornalístico, gerando a matriz tagged_j, incluindo apenas palavras e caracteres com classificações válidas:

In [ ]:
doc_j = cogroo.analyze(str(corpus_j.raw()))

sents_j = []

for st_j in doc_j.sentences:
    sents_j=sents_j+st_j.tokens

tagged_j = []

for tokens in sents_j:
    aux = re.split(r'#',str(tokens))
    aux2 = re.split(r' ',aux[1])
    if(aux2[0] in classes_cogroo):
        tagged_j = tagged_j + [(aux[0],aux2[0],aux2[1])]

print(tagged_j, end='')

In [ ]:
doc = cogroo.analyze(texto_jornalistico)

sents = []

for st in doc.sentences:
    sents=sents+st.tokens   

tagged_j = []

for tokens in sents:
    aux = re.split(r'#',str(tokens))
    aux2 = re.split(r' ',aux[1])
    if(aux2[0] in classes_cogroo):
        tagged_j = tagged_j + [(aux[0],aux2[0],aux2[1])]

doc = cogroo.analyze(texto_jornalistico_pt2)

sents = []

for st in doc.sentences:
    sents=sents+st.tokens   

for tokens in sents:
    aux = re.split(r'#',str(tokens))
    aux2 = re.split(r' ',aux[1])
    if(aux2[0] in classes_cogroo):
        tagged_j = tagged_j + [(aux[0],aux2[0],aux2[1])]

print(tagged_j, end='')


### Análise morfológica do corpus científico
Análise do cogroo do corpus científico, gerando a matriz tagged_c, incluindo apenas palavras e caracteres com classificações válidas:

In [ ]:
doc_c = cogroo.analyze(str(corpus_c.raw()))

sents_c = []

for st_c in doc_c.sentences:
    sents_c=sents_c+st_c.tokens

tagged_c = []

aux=''
aux2=''
for tokens in sents_c:
    aux = re.split(r'#',str(tokens))
    aux2 = re.split(r' ',aux[1])
    if(aux2[0] in classes_cogroo):
        tagged_c = tagged_c + [(aux[0],aux2[0],aux2[1])]

print(tagged_c, end='')

## Geração dos gráficos de distribuição de frequência


In [ ]:
# Importação das ferramentas necessárias
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

### Frequência de cada classe morfológica

#### Texto jornalístico

In [ ]:
#Criacao de uma string classificações
classificacoes_j = ''

#Colocar nessa string a classificacao morfologica de cada palavra
# na matriz etiquetada do corpus
i = 0
for x in tagged_j:
    classificacoes_j  = classificacoes_j + tagged_j[i][1] + ' '
    i = i + 1

#Tokenizar essa string, sendo identificada cada palavra/classificação
# como elemento da lista tokens
tokens_j = word_tokenize(classificacoes_j)

#Usar o dicionário pos para traduzir as classificações
# para um formato legível
i = 0
for x in tokens_j:
    tokens_j[i] = pos[tokens_j[i]]
    i = i + 1

#Gerar e apresentar a distribuição de frequência
fDist_class_j = FreqDist(word for word in tokens_j)

fDist_class_j

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nltk

x=fDist_class_j.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Frequência de classes morfológicas - jornalístico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Classes')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()


plt.show()

#### Texto científico

In [ ]:
#Criacao de uma string classificações
classificacoes_c = ''

#Colocar nessa string a classificacao morfologica de cada palavra
# na matriz etiquetada do corpus
i = 0

for x in tagged_c:
    if not ((tagged_c[i][0]=='São' and tagged_c[i+1][0]=='Paulo')
        or (tagged_c[i][0]=='São' and tagged_c[i+1][0]=='Bernardo')):
            classificacoes_c  = classificacoes_c + tagged_c[i][1] + ' '
    i = i + 1

#Tokenizar essa string, sendo identificada cada palavra/classificação
# como elemento da lista tokens
tokens_c = word_tokenize(classificacoes_c)

#Usar o dicionário pos para traduzir as classificações
# para um formato legível
i = 0
for x in tokens_c:
    tokens_c[i] = pos[tokens_c[i]]
    i = i + 1

#Gerar e apresentar a distribuição de frequência
fDist_class_c = FreqDist(word for word in tokens_c)

fDist_class_c

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nltk

x=fDist_class_c.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Frequência de classes morfológicas - científico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Classes')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()


plt.show()

### Substantivos mais comuns

#### Texto jornalístico

In [ ]:
#Criacao de uma string da classe
substantivos_j = ''

#Colocar nessa string as palavras lematizadas que correspondem
# à classe desejada
i = 0
for x in tagged_j:
    if (pos[ tagged_j[i][1] ] == 'substantivo'):
        substantivos_j  += cogroo.lemmatize( tagged_j[i][0] ) + ' '
    i = i + 1

#Tokenizar a string
tokens_j = word_tokenize(substantivos_j)

#Gerar a distribuição de frequência
fDist_subst_j = FreqDist(word for word in tokens_j)

fDist_subst_j

In [ ]:
x=fDist_subst_j.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Substantivos mais comuns - jornalístico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Substantivo primitivo')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()


plt.show()

#### Texto científico

In [ ]:
#Criacao de uma string classificações
substantivos_c = ''

#Colocar nessa string a classificacao morfologica de cada palavra
# na matriz etiquetada do corpus
i = 0
for x in tagged_c:
    if (pos[ tagged_c[i][1] ] == 'substantivo'):
        substantivos_c  += cogroo.lemmatize( tagged_c[i][0] ) + ' '
    i = i + 1

#Tokenizar essa string, sendo identificada cada palavra/classificação
# como elemento da lista tokens
tokens_c = word_tokenize(substantivos_c)

#Gerar e apresentar a distribuição de frequência
fDist_subst_c = FreqDist(word for word in tokens_c)

fDist_subst_c

In [ ]:
x=fDist_subst_c.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Substantivos mais comuns - científico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Substantivo primitivo')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()


plt.show()

### Adjetivos mais comuns

#### Texto jornalístico

In [ ]:
#Criacao de uma string classificações
adjetivos_j = ''

#Colocar nessa string a classificacao morfologica de cada palavra
# na matriz etiquetada do corpus
i = 0
for x in tagged_j:
    if (pos[ tagged_j[i][1] ] == 'adjetivo'):
        adjetivos_j  += cogroo.lemmatize( tagged_j[i][0] ) + ' '
    i = i + 1

#Tokenizar essa string, sendo identificada cada palavra/classificação
# como elemento da lista tokens
tokens_j = word_tokenize(adjetivos_j)

#Gerar e apresentar a distribuição de frequência
fDist_adj_j = FreqDist(word for word in tokens_j)

fDist_adj_j

In [ ]:
x=fDist_adj_j.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Adjetivos mais comuns - jornalístico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Adjetivo primitivo')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()


plt.show()

#### Texto científico

In [ ]:
#Criacao de uma string classificações
adjetivos_c = ''

#Colocar nessa string a classificacao morfologica de cada palavra
# na matriz etiquetada do corpus
i = 0
for x in tagged_c:
    if (pos[ tagged_c[i][1] ] == 'adjetivo'):
        adjetivos_c  += cogroo.lemmatize( tagged_c[i][0] ) + ' '
    i = i + 1

#Tokenizar essa string, sendo identificada cada palavra/classificação
# como elemento da lista tokens
tokens_c = word_tokenize(adjetivos_c)

#Gerar e apresentar a distribuição de frequência
fDist_adj_c = FreqDist(word for word in tokens_c)

fDist_adj_c

In [ ]:
x=fDist_adj_c.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Adjetivos mais comuns - científico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Adjetivo primitivo')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()
plt.show()

### Verbos mais comuns

#### Texto jornalístico

In [ ]:
#Criacao de uma string classificações
verbos_j = ''

#Colocar nessa string a classificacao morfologica de cada palavra
# na matriz etiquetada do corpus
i = 0
for x in tagged_j:
    if (pos[ tagged_j[i][1] ] == 'verbo finitivo' 
    and tagged_j[i+1][0]!='Paulo'):
        verbos_j  += cogroo.lemmatize( tagged_j[i][0] ) + ' '
    i = i + 1

#Tokenizar essa string, sendo identificada cada palavra/classificação
# como elemento da lista tokens
tokens_j = word_tokenize(verbos_j)

#Gerar e apresentar a distribuição de frequência
fDist_verb_j = FreqDist(word for word in tokens_j)

fDist_verb_j

In [ ]:
x=fDist_verb_j.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Verbos mais comuns - jornalístico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Verbo primitivo')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()


plt.show()

#### Texto científico

In [ ]:
#Criacao de uma string classificações
verbos_c = ''

#Colocar nessa string a classificacao morfologica de cada palavra
# na matriz etiquetada do corpus
i = 0
for x in tagged_c:
    if pos[ tagged_c[i][1] ] == 'verbo finitivo' 
    and tagged_c[i+1][0]!='Paulo':
        verbos_c  += cogroo.lemmatize( tagged_c[i][0] ) + ' '
    i = i + 1

#Tokenizar essa string, sendo identificada cada palavra/classificação
# como elemento da lista tokens
tokens_c = word_tokenize(verbos_c)

#Gerar e apresentar a distribuição de frequência
fDist_verb_c = FreqDist(word for word in tokens_c)

fDist_verb_c

In [ ]:
x=fDist_verb_c.most_common(20)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

plt.rcParams["figure.figsize"] = [16,8]

matplotlib.rcParams.update({'font.size': 15})

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Ocorrências')

ax.set_title('Verbos mais comuns - científico')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Verbo primitivo')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)

fig.tight_layout()

plt.show()

##  Gráficos sem Stopwords

In [ ]:
#teste de stop words em português
import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords
stopwords=nltk.corpus.stopwords.words('portuguese')
stopwords[:100]

In [ ]:
#Importação de stop words em português
import nltk
nltk.download('stopwords') 
from nltk.corpus import stopwords
stopwords=nltk.corpus.stopwords.words('portuguese')

# distribuição de Frequencia do texto jornalístico
# sem stopwords
fd_j2 = nltk.FreqDist(w.lower() for w
 in corpus_j.words() if w not in stopwords)

fd_j2.most_common(30)

In [ ]:
#Frequencia de palavaras do corpus_j que não são stopwords
def content_fraction(corpus_j):
 content = [w for w in corpus_j 
 if w.lower() not in stopwords]
 return len(content) / len(corpus_j)
content_fraction(corpus_j.words())

In [ ]:
fd_j2
x=fd_j2.most_common(30)
types=[w for (w,f) in x]
freq=[f for (w,f) in x]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nltk

x = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars
fig, ax = plt.subplots()

rects1 = ax.bar(x,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Frequência')

ax.set_title('Frequência x Types')
ax.set_xticks(x)
ax.set_xticklabels(types)
ax.set_xlabel('Types')
ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()

plt.show()

In [ ]:
# distribuição de Frequencia do artigos científico
# sem stopwords
fd_c2 = nltk.FreqDist(w.lower() for w in 
corpus_c.words() if w not in stopwords)
fd_c2.most_common(30)

In [ ]:
#Frequencia de palavras do corpus_c que não são stopwords
def content_fraction(corpus_c):
 content = [w for w in corpus_c if w.lower() not in stopwords]
 return len(content) / len(corpus_c)
content_fraction(corpus_c.words())

In [ ]:
fd_c2
y=fd_c2.most_common(30)
types=[w for (w,f) in y]
freq=[f for (w,f) in y]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import nltk

y = np.arange(len(types))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(y,freq, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Frequência')

ax.set_title('Frequência x Types')
ax.set_xticks(y)
ax.set_xticklabels(types)
ax.set_xlabel('Types')

ax.legend()
ax.tick_params(axis ='x', rotation = 90)
fig.tight_layout()

plt.show()

## Análise de palavras chave dos textos jornalísticos

In [ ]:
import nltk
from nltk import ngrams
tks=corpus_j.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

ngramas

# Frequencia de palavras com SER no jornalistico

In [ ]:
tks=corpus_j.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

palavras = ''

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(bigramas)):
  if cogroo.lemmatize(bigramas[i][0]) == 'ser' 
    and bigramas[i+1][0]!='Paulo' 
    and bigramas[i+1][0]!='Bernardo':
      palavras = palavras + cogroo.lemmatize(bigramas[i][1]) + ' '
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1


palavras_tokens = word_tokenize(palavras)

fDist_palavras = FreqDist(word.lower() for word in palavras_tokens)

fDist_palavras

# Frequencia de palavras com SER no cientifico

In [ ]:
tks=corpus_c.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

palavras = ''

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(bigramas)):
  if cogroo.lemmatize(bigramas[i][0]) == 'ser' 
    and bigramas[i+1][0]!='Paulo' 
    and bigramas[i+1][0]!='Bernardo':
      palavras = palavras + cogroo.lemmatize(bigramas[i][1]) + ' '
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1


palavras_tokens = word_tokenize(palavras)

fDist_palavras = FreqDist(word.lower() for word in palavras_tokens)

fDist_palavras

# Frequencia de palavras com IR no jornalistico

In [ ]:
tks=corpus_j.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

palavras = ''

types=list(set(tks))

contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(bigramas)):
  if cogroo.lemmatize(bigramas[i][0]) == 'ir':
        palavras = palavras + cogroo.lemmatize(bigramas[i][1]) + ' '
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1


palavras_tokens = word_tokenize(palavras)

fDist_palavras = FreqDist(word.lower() for word in palavras_tokens)

fDist_palavras

# Frequencia de palavras com IR no cientifico

In [ ]:
tks=corpus_c.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

palavras = ''

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(bigramas)):
  if cogroo.lemmatize(bigramas[i][0]) == 'ir':
        palavras = palavras + cogroo.lemmatize(bigramas[i][1]) + ' '
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1


palavras_tokens = word_tokenize(palavras)

fDist_palavras = FreqDist(word.lower() for word in palavras_tokens)

fDist_palavras

# Exemplos de frases com verbo IR: Jornalistico

In [ ]:
tks=corpus_j.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(ngramas)):
  if cogroo.lemmatize(ngramas[i][0]) == 'ir':
    print(ngramas[i])
    contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
    contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1

# Exemplos de frases com verbo IR: Científico

In [ ]:
tks=corpus_c.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(ngramas)):
  if cogroo.lemmatize(ngramas[i][0]) == 'ir':
    print(ngramas[i])
    contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
    contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1

# Exemplos de frases com verbo SER: Jornalistico

In [ ]:
tks=corpus_j.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(ngramas)):
  if cogroo.lemmatize(ngramas[i][0]) == 'ser' 
    and ngramas[i+1][0]!='Paulo' 
    and ngramas[i+1][0]!='Bernardo':
      print(ngramas[i])
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1

# Exemplos de frases com verbo SER: Científico

In [ ]:
tks=corpus_c.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(ngramas)):
  if cogroo.lemmatize(ngramas[i][0]) == 'ser' 
    and ngramas[i+1][0]!='Paulo' 
    and ngramas[i+1][0]!='Bernardo':
      print(ngramas[i])
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
      contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1

# Análises adicionais: Palavras associadas a Zona, Avenida e Rua

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.probability import FreqDist


tks=corpus_j.words()
ngramas = list(ngrams(tks, 4))
bigramas = list(ngrams(tks, 2))

palavras = ''

types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(bigramas)):
  if bigramas[i][0] == 'zona':
        print(bigramas[i])
        palavras = palavras + bigramas[i][1] + ' '
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1

        palavras_tokens = word_tokenize(palavras)

fDist_palavras = FreqDist(word.lower() for word in palavras_tokens)

fDist_palavras

In [ ]:
types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(ngramas)):
  if ngramas[i][0] == 'avenida':
        print(ngramas[i])
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1

In [ ]:
types=list(set(tks))
contagem=np.zeros((len(types),len(types)))
dicio={}
for i in range(len(types)):
  dicio[types[i]]=i
for i in range(len(ngramas)):
  if ngramas[i][0] == 'rua':
        print(ngramas[i])
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][0]]] += 1
        contagem[dicio[ngramas[i][1]]][dicio[ngramas[i][2]]] += 1

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=162f482c-e397-45e6-9a74-fda6ea1a664a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>